Using the [QMENTA SDK](https://www.qmenta.com/sdk/) you can easily package and deploy your predictive model to the QMENTA Platform in order to run it in a scalable cloud environment.
You can learn more about the SDK from a technical perspective at https://docs.qmenta.com/sdk/.

If you registered to the QMENTA Platform using the **promotional code** shown in [1.qmenta_client_download_data.ipynb](./1.qmenta_client_download_data.ipynb), you already have **developer access** to the platform. 

With this level of access you can create, edit and delete **analysis tools** in the platform. You will first neeed to package your applications into a Docker container image, and then upload it to [Docker Hub](https://hub.docker.com/) so that the platform can fetch it and securely run it.

![My Tools view](./assets/my_tools.png)

In this Jupyter notebook we will show you how to:
- Write an script that does the prediction and is integrated with the QMENTA Platform via the QMENTA SDK
- Build a Docker image that packages everything needed to perform inference
- Upload the produced Docker image to Docker Hub
- Register our tool in **My Tools**
- Run the tool on one subject in the platform

# 1. Prepare the SDK script

Our script will basically load the persisted model from disk, load the data to be predicted from disk and run inference on this model using the loaded data. On top of that, the script will communicate with the platform in order to download the input data, set the progress of the analysis and finally upload the results back to the platform, so that they can be accessed via the web interface.

A possible implementation of such behaviourcan be found in the [predict_age.py](./predict_age.py) script.

In order to understand the mechanics of SDK tool development please check this [documentation page](https://docs.qmenta.com/sdk/using_sdk.html).

# 2. Build the Docker Image

### Install Docker

Head over to [Get Docker](https://docs.docker.com/install/#desktop) documentation page and follow the **Docker Community Edition** installation process for your platform of choice.

### Use a Dockerfile

A Dockerfile allows you to programatically specify how you build the Docker images without having to manually modify a container and create a snapshot of it.

You can check the example [Dockerfile](./Dockerfile) included in this repository. For more details about the Dockefile syntax check the documentation [here](https://docs.docker.com/engine/reference/builder/).

### Build the image

We will use the `docker build` command to build our Docker image, that later we will push to the default public Docker registry (DockerHub)

First open a terminal and type the following commands:
```
cd ~/qmenta_cnic_1000_brains_challenge/
mkdir docker_build
```

In this `docker_build` folder we will put all the necessary files to package our application as a Docker image. Assuming that you have cloned this repository in `~/1000_brains_challenge`:
```
cp ~/1000_brains_challenge/Dockerfile ./docker_build/Dockerfile
cp ~/1000_brains_challenge/predict_age.py ./docker_build/predict_age.py
cp ~/1000_brains_challenge/environment.yml ./docker_build/environment.yml
cp ~/qmenta_cnic_1000_brains_challenge/models/linear_regression_example.pkl ./docker_build/linear_regression_example.pkl
```

Finally we will run the docker build command:
```
docker build -t 1000_brains_example ./docker_build
```

# 3. Upload your Docker image to DockerHub

First you will need to [create an account in DockerHub](https://hub.docker.com/). By creating an account you will have the possibility to upload all your images to DockerHub and make them available to anybody. The [Free Account plan](https://hub.docker.com/billing-plans/) also includes one private image, which can only be accessed with your credentials.

Now we will tag the image we just created so that you can upload it to you newly created DockerHub account. If your username is **user**, the you will have to run the following commands:

```
docker tag 1000_brains_example user/1000_brains_challenge:1.0
```

Then to push your image just run:

```
docker login -u user
```

You will be prompted to introduce your DockerHub account's password, and you will see a `Login Succeeded` message after successfully introducing your password.

Finally, you will be able to push your image by executing:

```
docker push user/1000_brains_challenge:1.0
```

If you access DockerHub you should see the following page:

![DockerHub image](assets/dockerhub_image.png)

By default your Docker image will be made **public**, which means that anybody can pull it without any credentials, like so:
```
docker pull user/1000_brains_challenge:1.0
```

However you can make it private in order to protect your work: go to **Settings** in your DockerHub repository and in **Visibility Settings** you can click the **Make Private** button.

# 4. Register your tool

Follow the detailed intructions in our [SDK Documentation page](https://docs.qmenta.com/sdk/registering.html) in order to register your tool in the QMENTA platform.

The **My Tools** form should be filled with the following specification:
- **Name**: 1000 Brains Challenge - TeamID
- **Short Name**: 1000_brains_challenge_teamid
- **Analysis Code**: 1000_brains_challenge_teamid
- **Repository URL**: https://hub.docker.com
- **Num. of cores**: 1
- **RAM limit in GB**: 2

![My Tools Registration form](assets/tool_registration_form.png)

And the **Settings (Editor)** tab should be filled with information found in [predict_age_settings.json](predict_age_settings.json):

![Tool Settings Form](assets/tool_settings_form.png)

# 5. Run the tool in the platform

Once the tool has been registered in the platform you can go to **My Analysis** and click **Start analysis**, which will prompt the following form:

![Start SDK Tool](assets/start_sdk_tool.png)

In this form you only need to select the ANTs Analysis that you want to use as input and then select **Start Analysis**. The dashboard will now show the analysis as pending (grey dot on the left side) and after scheduling to the appropriate execution machine the analysis will start to run (blue dot on the left side).

![Run SDK Tool](assets/run_sdk_tool.png)

Finally the tool will complete successfully and give you the prediction in the form of a text file called **age.txt**

![SDK Tool Result](assets/sdk_tool_results.png)

# 6. (Advanced) Testing the tool locally

You can use the following [guide](https://docs.qmenta.com/sdk/testing.html) in our SDK Documentation page to get started with local testing and debugging of your tools, without the need of going over the whole process of deploying the built image and running an analysis in the QMENTA platform to test how the tool behaves.